In [ ]:
#Ce travail était fait par OUCOUC Hafid,Maouche Massinissa et Dinel Debib
import pandas as pd
import numpy as np
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.exceptions import ConvergenceWarning
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.preprocessing import OneHotEncoder


# Ignorer les avertissements de convergence
warnings.filterwarnings('ignore', category=ConvergenceWarning)

# ***Chargement des données et préparation***

In [ ]:
# Chargement des données
data = pd.read_csv('/content/credit.data', sep='\t', header=None)

# Séparation des caractéristiques et de la cible
X = data.iloc[:, :-1]  # Caractéristiques
y = data.iloc[:, -1]   # Cible

# Identification des colonnes numériques
numeric_columns = [1, 2, 7, 10, 13, 14]  # Colonnes qui devraient être numériques

# Remplacer '?' par NaN et convertir en float
for col in numeric_columns:
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Suppression des lignes avec des valeurs NaN
X_numeric = X[numeric_columns].dropna()

# Mise à jour de la cible en fonction des lignes supprimées
y = y[X_numeric.index]

# Vérification des données
print("X_numeric après traitement des valeurs manquantes:")
print(X_numeric.head())
print("Taille de l'échantillon:", X_numeric.shape)
print("Nombre d'exemples positifs:", (y == '+').sum())
print("Nombre d'exemples négatifs:", (y == '-').sum())

# Binarisation de la cible
lb = LabelBinarizer()
y_binary = lb.fit_transform(y).ravel()  # Convertit '+' en 1 et '-' en 0

# Fonction pour exécuter les classificateurs
def run_classifiers(clfs, X, y):
    for clf_name, clf in clfs.items():
        scores = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
        print(f"{clf_name}: AUC = {np.mean(scores):.2f} (+/- {np.std(scores):.2f})")

# Autres classificateurs pour la comparaison
rf_classifier = RandomForestClassifier()
logistic_regression = LogisticRegression(max_iter=1000)  # Augmentation du nombre d'itérations
knn_classifier = KNeighborsClassifier()
svm_classifier = SVC(probability=True)  # probability=True pour permettre l'AUC

# Ajout des classificateurs au dictionnaire
classifiers = {
    'RandomForest': rf_classifier,
    'LogisticRegression': logistic_regression,
    'KNN': knn_classifier,
    'SVM': svm_classifier
}

run_classifiers(classifiers, X_numeric, y_binary)

X_numeric après traitement des valeurs manquantes:
      1      2     7   10     13   14
0  30.83  0.000  1.25   1  202.0    0
1  58.67  4.460  3.04   6   43.0  560
2  24.50  0.500  1.50   0  280.0  824
3  27.83  1.540  3.75   5  100.0    3
4  20.17  5.625  1.71   0  120.0    0
Taille de l'échantillon: (666, 6)
Nombre d'exemples positifs: 299
Nombre d'exemples négatifs: 367
RandomForest: AUC = 0.84 (+/- 0.05)
LogisticRegression: AUC = 0.83 (+/- 0.06)
KNN: AUC = 0.70 (+/- 0.02)
SVM: AUC = 0.70 (+/- 0.04)


# ***Normalisation des variables continues***

In [ ]:
# Normalisation des données avec StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)

# Ou, normalisation des données avec MinMaxScaler
# scaler = MinMaxScaler()
# X_scaled = scaler.fit_transform(X_numeric)

# Exécuter les classificateurs avec les données normalisées
run_classifiers(classifiers, X_scaled, y_binary)


RandomForest: AUC = 0.84 (+/- 0.05)
LogisticRegression: AUC = 0.82 (+/- 0.06)
KNN: AUC = 0.81 (+/- 0.06)
SVM: AUC = 0.83 (+/- 0.06)


# ***Traitement de données manquantes***

In [ ]:
# Chargement des données
data = pd.read_csv('/content/credit.data', sep='\t', header=None)

# Séparation des caractéristiques et de la cible
X = data.iloc[:, :-1].values  # Caractéristiques
y = data.iloc[:, -1].values   # Cible

# Identification des colonnes catégorielles et numériques
col_cat = [0, 3, 4, 5, 6, 8, 9, 11, 12]  # Indices des colonnes catégorielles
col_num = [i for i in range(X.shape[1]) if i not in col_cat]  # Indices des colonnes numériques

# Conversion des variables catégorielles en valeurs numériques
label_encoders = []
X_cat = np.copy(X[:, col_cat])
for col_id in range(X_cat.shape[1]):
    le = LabelEncoder()
    X_cat[:, col_id] = le.fit_transform(X_cat[:, col_id].astype(str))
    label_encoders.append(le)

# Imputation pour les variables catégorielles
imp_cat = SimpleImputer(missing_values=0, strategy='most_frequent')
X_cat = imp_cat.fit_transform(X_cat)

# Traitement des variables numériques
X_num = np.copy(X[:, col_num])
X_num[X_num == '?'] = np.nan
X_num = X_num.astype(float)

# Imputation pour les variables numériques
imp_num = SimpleImputer(missing_values=np.nan, strategy='mean')
X_num = imp_num.fit_transform(X_num)

# Fusion des variables catégorielles et numériques
X_combined = np.hstack((X_cat, X_num))

# Binarisation de la cible
lb = LabelBinarizer()
y_binary = lb.fit_transform(y).ravel()

# Définition des classificateurs
classifiers = {
    'RandomForest': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(probability=True)
}

# Fonction pour exécuter les classificateurs
def run_classifiers(clfs, X, y):
    for clf_name, clf in clfs.items():
        scores = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
        print(f"{clf_name}: AUC = {np.mean(scores):.2f} (+/- {np.std(scores):.2f})")


# Exécution des classificateurs sur le jeu de données imputé
run_classifiers(classifiers, X_combined, y_binary)


RandomForest: AUC = 0.86 (+/- 0.05)
LogisticRegression: AUC = 0.82 (+/- 0.05)
KNN: AUC = 0.72 (+/- 0.04)
SVM: AUC = 0.69 (+/- 0.05)


# ***Traitement de variables catégorielles***

In [ ]:
# Encodage One-Hot pour les variables catégorielles
onehot_encoder = OneHotEncoder()
X_cat_bin = onehot_encoder.fit_transform(X_cat).toarray()

# Fusion des variables catégorielles encodées et numériques
X_combined_bin = np.hstack((X_cat_bin, X_num))

# Exécution des classificateurs sur le jeu de données avec encodage one-hot
run_classifiers(classifiers, X_combined_bin, y_binary)

RandomForest: AUC = 0.85 (+/- 0.05)
LogisticRegression: AUC = 0.83 (+/- 0.06)
KNN: AUC = 0.71 (+/- 0.05)
SVM: AUC = 0.69 (+/- 0.05)


# ***Construction du jeu de données***

In [ ]:
# Normalisation des variables numériques
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)

# Encodage One-Hot pour les variables catégorielles
onehot_encoder = OneHotEncoder()
X_cat_bin = onehot_encoder.fit_transform(X_cat).toarray()

# Construction du jeu de données final
X_final = np.hstack((X_cat_bin, X_num_scaled))

# Définition et exécution des classificateurs sur le jeu de données final
run_classifiers(classifiers, X_final, y_binary)


RandomForest: AUC = 0.84 (+/- 0.05)
LogisticRegression: AUC = 0.83 (+/- 0.06)
KNN: AUC = 0.76 (+/- 0.05)
SVM: AUC = 0.84 (+/- 0.06)
